In [2]:
# modules
import numpy as np
import pandas as pd
from pathlib import Path
import os
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, accuracy_score

In [3]:
# change dir
%cd /home/hirano/work/Competition/Nishika_cable/src/
%load_ext autoreload
%autoreload 2
!pwd

/home/hirano/work/Competition/Nishika_cable/src
/home/hirano/work/Competition/Nishika_cable/src


In [4]:
os.makedirs('../output/ensemble', exist_ok=True)

pref = '13' #!!Please change Submit Number !!

versions = ['006', '007', '008', '009', '010', '030', '035', '037', '039', '043', '044', '045', '046']

In [5]:
# Load
for i, version in enumerate(versions):
    oof_name = [x for x in os.listdir(f'../output/results/') if f'oof_{version}' in x]
    print(oof_name)

    if i==0:
        oof = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'pred_{i}'] = oof.pred.values
        oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        sub = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'pred_{i}'] = sub.target.values
        sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] = 0.33*sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

    else:
        _tmp = pd.read_csv(f'../output/results/{oof_name[0]}').sort_values('filename')
        oof[f'pred_{i}'] = _tmp.pred.values
        oof[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

        _tmp = pd.read_csv(f'../output/results/sub_{version}.csv')
        sub[f'pred_{i}'] = _tmp.target.values
        sub[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4',
             'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9',
             'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']] += 0.33*_tmp[['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']].values

oof = oof.reset_index(drop=True)

['oof_010_0.9868.csv']
['oof_044_0.9876.csv']
['oof_045_0.9863.csv']
['oof_046_0.8184.csv']


In [6]:
oof.head()

,filename,Xmin_Ymin_Xmax_Ymax,class,filename_trim,fold,target,pred,conf_0,conf_1,conf_2,...,conf_9,conf_10,conf_11,conf_12,conf_13,conf_14,pred_0,pred_1,pred_2,pred_3
0,00037f39.jpg,NaN,Lightning,00037f39_trim.jpg,1,3,3,0.000986,0.000755,0.000307,...,0.000279,0.000173,0.000882,0.001700,0.000197,0.000108,3,3,3,3
1,000c15d0.jpg,"[906, 1686, 1294, 1946]",USB_Type_C,NaN,2,8,8,0.000270,0.000284,0.000370,...,0.000163,0.000069,0.000167,0.004043,0.000103,0.000122,8,8,8,8
2,000f21ac.jpg,NaN,USB_Type_C,000f21ac_trim.jpg,3,8,8,0.000277,0.000260,0.000762,...,0.000362,0.000061,0.001423,0.001002,0.000501,0.000568,8,8,8,8
3,0013fa17.jpg,"[1458, 1756, 2304, 2602]",DisplayPort,NaN,0,14,14,0.004393,0.003100,0.004732,...,0.003053,0.001793,0.002434,0.000403,0.005107,1.089177,14,14,14,14
4,00150bc1.jpg,"[1181, 1692, 1658, 1985]",USB_Type_C,NaN,2,8,8,0.000601,0.000166,0.000687,...,0.000178,0.000069,0.000234,0.001156,0.000193,0.001779,8,8,8,8


In [7]:
sub.head()

,target,conf_0,conf_1,conf_2,conf_3,conf_4,conf_5,conf_6,conf_7,conf_8,conf_9,conf_10,conf_11,conf_12,conf_13,conf_14,pred_0,pred_1,pred_2,pred_3
0,8,0.002166,0.059658,0.001356,0.000927,0.026505,0.003278,0.000619,0.001499,1.214265,0.001172,0.000136,0.001733,0.003665,0.002089,0.000932,8,8,8,8
1,4,0.000380,0.016259,0.000104,0.002284,1.291576,0.002306,0.000399,0.001681,0.001302,0.000336,0.000828,0.001084,0.000857,0.000245,0.000390,4,4,4,4
2,8,0.000317,0.017118,0.000121,0.002969,0.251467,0.000228,0.000096,0.001511,1.041981,0.000093,0.000261,0.000301,0.003175,0.000182,0.000180,8,8,8,4
3,4,0.001303,0.003404,0.000378,0.005030,0.955078,0.001758,0.000305,0.001354,0.325120,0.000238,0.000437,0.000486,0.024064,0.000520,0.000480,4,4,4,8
4,1,0.001217,1.081072,0.000112,0.003674,0.190118,0.004640,0.000200,0.001730,0.028132,0.000238,0.001226,0.000918,0.006348,0.000158,0.000211,1,1,1,4


In [8]:
params = {
    'objective': 'multiclass',
    'num_class' : 15,
    'metrics': 'multi_error',
    'n_estimators': 10000,
    'boosting_type': 'gbdt',
    'num_leaves': 32,
    'max_depth': 2,
    'learning_rate': 0.001,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.3,
    'bagging_freq': 5,
    'reg_alpha': 0.5,
    'reg_lambda': 0,
}

In [9]:
stacking_oof = np.zeros(len(oof))
#stacking_sub = np.zeros(len(sub))
stacking_sub = []

features = [f'pred_{i}' for i in range(len(versions))]
features.extend(['conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4','conf_5',
                 'conf_6', 'conf_7', 'conf_8', 'conf_9','conf_10', 'conf_11',
                 'conf_12', 'conf_13', 'conf_14'])
print(features)

def lgb_f1_score(y_hat,data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1', f1_score(y_true, y_hat,average='macro'), True

n_fold = 5

for fold in range(n_fold):
    print(f'{fold=}')

    trn = oof[oof.fold!=fold]
    val = oof[oof.fold==fold]

    val_idx = val.index

    trn_x = trn[features]
    trn_y = trn['target']
    val_x = val[features]
    val_y = val['target']

    tst_x = sub[features]

    model = LGBMClassifier(**params)
    model.fit(
        trn_x,
        trn_y,
        eval_metric='multi_logloss',
        eval_set=[(val_x, val_y)],
        verbose=100,
        early_stopping_rounds=300
    )
    
    val_pred = model.predict(val_x)
    #stacking_sub += model.predict(tst_x)
    stacking_sub.append(model.predict(tst_x))
    
    stacking_oof[val_idx] = val_pred
    #break

['pred_0', 'pred_1', 'pred_2', 'pred_3', 'conf_0', 'conf_1', 'conf_2', 'conf_3', 'conf_4', 'conf_5', 'conf_6', 'conf_7', 'conf_8', 'conf_9', 'conf_10', 'conf_11', 'conf_12', 'conf_13', 'conf_14']
fold=0
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.3, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3
Training until validation scores don't improve for 300 rounds
[100]	valid_0's multi_logloss: 0.283943	valid_0's multi_error: 0.00870647
[200]	valid_0's multi_logloss: 0.0820998	valid_0's multi_error: 0.00746269
[300]	valid_0's multi_logloss: 0.0505667	valid_0's multi_error: 0.00621891
[400]	valid_0's multi_logloss: 0.0448464	valid_0's multi_error: 0.00746269
[500]	valid_0's multi_logloss: 0.0421255	valid_0's multi_error: 0.00870647
Early stoppi

# OOF

In [10]:
oof.pred = stacking_oof
print(oof.pred)

score = f1_score(oof.target.values, oof.pred.values, average='micro')
print(f'Score is {score:.4f}')

sub_name = '_'.join(versions)
oof.to_csv(f'../output/ensemble/{pref}_oof_stacking_{sub_name}_{score:.4f}.csv', index=False)

0        3.0
1        8.0
2        8.0
3       14.0
4        8.0
        ... 
4014     4.0
4015     3.0
4016     6.0
4017     3.0
4018     4.0
Name: pred, Length: 4019, dtype: float64
Score is 0.9893


# Submission

In [12]:
label_dict = {
   0 : 'HDMI',
   1 : 'USB_Mini',
   2 : 'Dock',
   3 : 'Lightning',
   4 : 'USB_Micro_B',
   5 : 'Mini_DisplayPort',
   6 : 'RJ_45',
   7 : 'USB_Type_A',
   8 : 'USB_Type_C',
   9 : 'USB_Micro_B_3.1',
  10 : 'USB_Type_B',
  11 : 'Lightning_T',
  12 : 'USB_Micro_B_W',
  13 : 'VGA',
  14 : 'DisplayPort'
}

In [13]:
import scipy.stats as stats

#print(stacking_sub)
results = stats.mode(stacking_sub, axis=0).mode[0]
print(len(results))

# class num to class name
targets = []
for i in range(len(results)):
    targets.append(label_dict[results[i]])
sub['class'] = targets
print(len(targets))

test = pd.read_csv('../input/test_with_fold.csv')
sub['filename'] = test['filename']

sub_name = '_'.join(versions)
sub[['filename', 'class']].to_csv(f'../output/ensemble/{pref}_stacking_{sub_name}.csv', index=False)

2200
2200
